# Image Understanding Coursework : Introduction to OpenCV in python
#### David Griffiths &nbsp;&nbsp;&nbsp; david.griffiths.16@ucl.ac.uk









## Basic OpenCV functions

##### Import necessary libraries

In [ ]:
%matplotlib inline

In [ ]:
import cv2
import numpy as np
import pylab as plt

##### Load image

In [ ]:
file = 'images/newyork.jpg'

# Load image in BGR colorspace
img_color = cv2.imread(file, 1)
#Load image in grayscale
img_gray = cv2.imread(file, 0)

img_color_RGB = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)

# Plot images using matplotlib
images = [img_color, img_gray, img_color_RGB]
titles = ['Colored Image', 'Grayscale Image', 'RGB Color Image']
plt.figure(figsize=(20,10))
for i in xrange(3):
    plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

In [ ]:
# To write an image for specify the save directory and name:
new_file = 'images/newyork_grayscale.jpg'
# Write the image to disk
cv2.imwrite(new_file, img_gray)
print 'image successfully saved to: ' + new_file

##### Read and edit pixels

OpenCV images are stored in the form of numpy arrays. The numpy array has the dimensions (image height x image width x number of channels). The images can therefore be treated as standard numpy $n$ dimension arrays.

In [ ]:
print 'One channel grayscale image: ', img_gray.shape
print '3 Chanel BGR colorspace image: ', img_color.shape

To access a specific pixel value basic numpy array indexing can be used. For example, to return the intensity of pixel x=500, y= 1000 you can use:

In [ ]:
pixel = 500, 750

print img_gray[pixel]
print img_color[pixel]

Notice how the colour image has 3 channels and therefore three pixel intensities (Blue, Green, Red). Further indexing can reutrn a specific intensity i.e:

In [ ]:
blue = img_color[pixel][0]
green = img_color[pixel][1]
red = img_color[pixel][2]

**Task**: How would we create a simple 'for' loop to loop through every pixel in a given image.

In [ ]:
# Try task here

## Geometric Transformations

When carrying out geometric transformations of an image it is important to think of the image as a standard matrix. This way typical linear algebra can be used to perform transformations.

##### Scaling

In [ ]:
# First we will import a new image which which will allow us to easily see the result of the transformations.

img = cv2.imread('images/circle.jpg', 1)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
# OpenCV offers a resize function which can be called using cv2.resize. 
# We will go through a few ways to use this function.

rows, cols, channels = img.shape

# Stretch collumns by a factor of 2
img_resize = cv2.resize(img, (cols*2, rows), interpolation = cv2.INTER_CUBIC)

# Notice the interpolation method here. OpenCV offers multiple interpolation functions.
# In general it is reccommended:
# Shrinking: cv2.INTER_AREA
# Zooming: cv2.INTER_CUBIC (slow) or cv2.INTER_LINEAR
# Default (general): cv2.INTER_LINEAR

# Half the size of the image
img_resize = cv2.resize(img, (cols/2, rows/2), interpolation = cv2.INTER_AREA)

# Set to specified size
img_resize = cv2.resize(img, (180, 120),interpolation = cv2.INTER_LINEAR)

In [ ]:
plt.imshow(img_resize)

##### Translation

OpenCV provides a function (`cv2.warpAffine`) to carry out affine transformations. The main requirements for this are the original image you wish to apply the transformation and the transformation matrix $M$. We must therefore first compute the transformation matrix $M$. If you know the shift in ($x$, $y$) that you wish to compute you can let:  

$M= \begin{bmatrix} 1 & 0 & x \\ 0 & 1 & y \end{bmatrix}$

In [ ]:
M = np.float32([[1,0,25],[0,1,50]])
translation = cv2.warpAffine(img, M, (cols, rows))

In [ ]:
plt.imshow(translation)

Similarly a rotation matrix can be computed using an $M$ matrix following:

$M= \begin{bmatrix} \cos(\theta) & -\sin(\theta) \\ \cos(\theta) & \sin(\theta) \end{bmatrix}$

OpenCV provides a tool for creating these matrix with the function `cv2.getRotationMatrix2D`

In [ ]:
# Create rotation matrix for 25 degrees rotation in respect to the image center
M = cv2.getRotationMatrix2D((cols/2, rows/2), 25, 1)

# Create rotation matrix for 25 degrees rotation with different roation center
M_offset = cv2.getRotationMatrix2D((250, 250), 25, 1)

transformation = cv2.warpAffine(img, M, (cols, rows))
transformation_offset = cv2.warpAffine(img, M_offset, (cols, rows))

print 'M matrix \n', M, '\n\n'
print 'M offset matrix \n', M_offset

plt.figure(figsize=(7,7))
plt.subplot(221), plt.imshow(transformation)
plt.subplot(222), plt.imshow(transformation_offset)

##### Affine transformation

In affine transformations all parallel lines will remain parallel after the transformation. OpenCV provides a function `cv2.getAffineTransformation` to determine the transformation matrix. The minimum number of points for an affine transformation is 3. To determine the $M$ matrix we must therefore make 2 3x1 matrix with the first containing the original co-ordinates of the points, the second will contain the location of the points after the transformation.

In [ ]:
originalPts = np.float32([[50, 50], [100, 150], [180, 120]])
transformedPts = np.float32([[60, 75],[110, 190], [175, 100]])

# Determine M matrix
M = cv2.getAffineTransform(originalPts, transformedPts)

# Pass image and M matrix in cv2.warpAffine function
transformation = cv2.warpAffine(img, M, (cols, rows))

plt.figure(figsize=(7,7))
plt.subplot(221), plt.imshow(img)
plt.subplot(222), plt.imshow(transformation)

##### Perspective transformation

Perspective transformation is similar to affine however instead of preserving parallel lines it preserves collinearity and incidence. As perspective offers an extra degree of translation 4 points are required to determine the $M$ matrix. By providing the co-ordinates in the same manner as the affine transformation, the matrix can be computed using the function `cv2.getPerspectiveTransform`. The transormation can then be carried out using `cv2.warpPerspective`.

In [ ]:
originalPts = np.float32([[50, 50], [100, 150], [180, 120], [30, 30]])
transformedPts = np.float32([[50, 53], [98, 155], [175, 120], [27, 31]])

# Determine M matrix
M = cv2.getPerspectiveTransform(originalPts, transformedPts)

# Pass image and M matrix in cv2.warpAffine function
transformation = cv2.warpPerspective(img, M, (cols, rows))

plt.figure(figsize=(7,7))
plt.subplot(221), plt.imshow(img)
plt.subplot(222), plt.imshow(transformation)

## Image Thresholding

Basic thresholding in OpenCV is achieved through the `cv2.threshold` function. There are multiple options to determine how the threshold can be computed, but first we will create a simple binary threshold. 

**TASK:** Using the code we created earlier how could this be adapted into a simple binary classifier for a grayscale image with a critical value of 150?

In [ ]:
# First lets load and display an image in RGB colorspace
img = cv2.imread('images/ucl.jpg', 0)

plt.imshow(img, cmap='gray')

In [ ]:
# Attempt task here

We will now carry this out using OpenCV. The function cv2.THRESH_BINARY defines the threshold style. The availble styles are:

`cv2.THRESH_BINARY`

`cv2.THRESH_BINARY_INV`

`cv2.THRESH_TRUNC`

`cv2.THRESH_TOZERO`

`cv2.THRESH_TOZERO_INV`

In [ ]:
ret, thresh = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
ret, thresh_inv = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY_INV)

images = [img, thresh, thresh_inv]
titles = ['Original', 'Binary', 'Binary Inverse']
plt.figure(figsize=(20,10))
for i in xrange(len(images)):
    plt.subplot(3,3,i+1),plt.imshow(images[i],cmap='gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

Adaptive thresholding is used for images with varying lighting conditions. Instead of using a 1 constant value for the whole of the image we can split the image into regions and threshold each region individually based on the neighbourhood mean. This can give much more resonable results when the lighting changes greatly across an image (i.e. heavy shadows). We can also take the weighted sum of neighbourhood values where the weights are a gaussian window. This can be useful to reducing thresholds that are very noisy.

In [ ]:
adaptive_thresh = cv2.adaptiveThreshold(img,255, \
                            cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,15,7)

adaptive_thresh_g = cv2.adaptiveThreshold(img,255, \
                                cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,15,7)

plt.figure(figsize=(15,15))
plt.subplot(221), plt.imshow(adaptive_thresh, cmap='gray'), plt.xticks([]),plt.yticks([])
plt.subplot(222), plt.imshow(adaptive_thresh_g, cmap='gray'), plt.xticks([]),plt.yticks([])

##### Otsu's Binarisation

Otsu's binirisation is a method to automatically determine the threshold for a bimodal image. This attempts to find the optimum threshold value by converting the image to a histogram and finding the value in between the two main peaks. This method is therefore only suitable for bimodal images.

In [ ]:
ret, otsu = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

images = [img, thresh, adaptive_thresh, otsu]
titles = ['Original', 'Binary Threshold', 'Adaptive Threshold', 'Otsu Binarisation']
plt.figure(figsize=(20,10))
for i in xrange(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

##### RGB Thresholding

All the thresholding above works for grayscale images. However, sometimes the important details are related to the ratio of RGB colours, not just the overall intensity of the pixel. This is clearly visable in our New York image we loaded in earlier. The clearest and most obvious feature is Central Park.

In [ ]:
# First lets load the image back into memory

img = cv2.imread('images/newyork.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(90,50))
cv2.imwrite('90x50.jpg',img_rgb)

In [ ]:
critical_thresh = 50

seg_img = img_rgb.copy()

for x in range(0, seg_img.shape[0]):
    for y in range(0, seg_img.shape[1]):
        r,g,b = seg_img[x, y]
        if g < r*2 and g < b*2:
            seg_img[x, y] = [0, 0, 0]
     
    
# Now we can apply a binarisation to the image to extract only the bit we want

# First convert to grayscale

seg_img_gray = cv2.cvtColor(seg_img, cv2.COLOR_RGB2GRAY)

# Or as before we can apply an otsu binarisation
ret, otsu = cv2.threshold(seg_img_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

images = [img, seg_img, seg_img_gray, otsu]
titles = ['Original', 'RGB Threshold', 'Gray Scale RGB Threshold', 'Otsu Binarisation']
plt.figure(figsize=(20,10))
for i in xrange(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

Alternatively, we could achieve this faster by converting into Hue, Saturation, Value (HSV) color space. In general it is quicker to represent a specific colour in HSV. There are lots of resources online explaining HSV colorspace and why it is useful. Here, we first create a mask using the `cv2.inRange` function. Once we have our mask we can then use `cv2.bitwise_and` function to compute a per-element bit-wise conjunction of the two images. `cv2.wise_and` also has a `XOR`, `OR` and `NOT` function.

A simple trick for finding HSV values is to find the RGB value a representative the pixel, then make a 3x1 numpy array and convert it from RGB to HSV using cv2.cvtColor. You can then use this value as a centre point for the range by adding and subtracting a value to the HUE for the upper and lower limits to make them look like they do above. [Here](https://alloyui.com/examples/color-picker/hsv) is a useful link for finding HSV values.

Note: Because we are dealing with 8Bit images here the HUE range is stored between 0 and 180 instead of 0 and 360. Therefore, you will need to half the hue value you derive from color pickers.



In [ ]:
# First lets convert our image into HSV
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Then define the limits for green
lower_green = np.array([25, 50, 50])
upper_green = np.array([75,255,255])

# Use cv2.inRange function to mask green pixels
mask = cv2.inRange(hsv_img, lower_green, upper_green)

# Computer per-element bit-wise conjunction
res = cv2.bitwise_and(img_rgb, img_rgb, mask=mask)

plt.figure(figsize=(20,10))
plt.subplot(221), plt.title('HSV inRange mask'), plt.xticks([]),plt.yticks([]), plt.imshow(mask, cmap='gray')
plt.subplot(222), plt.title('Bitwise-AND mask'), plt.xticks([]),plt.yticks([]), plt.imshow(res, cmap='gray')

Have a think about how you could extract more than one color at a time, for example all strong red, green and blue features in an image. Have a go at this before next week. You can choose any image/colours you want, or you can try to extract the red, green and blue markings on the image below.

In [ ]:
# Attempt task here before next week
# Try to find your own image, if you are stuck you can use this one:

img = cv2.imread('images/sign.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

## Coursework

**Task**

Your task is to use OpenCV and Numpy tools to extract a specific feature from a single (or group of) image(s). The feature(s) of choice as well as image(s) is up to you. It can be imagery acquired from satellite, aerial or terrestrial methods as well as your choice of spectral bands (i.e. R, G, B, NIR, SWIR etc.). You are encouraged to think outside-the-box and use internet resources such as tutorials to incorporate new methods not discussed within these practicals. 

You should assess the accuracies of your proposed solution, highlighting the strengths and weaknesses. The accuracy assessment should be undertaken both on images you designed the solution around, as well as new images your system has not seen before. You should comment on why you think the solution is performing the way it is.

**Delivery**

The coursework comprises two components. First, a 2000 word research report style write up (i.e. Introduction, Methodology, Results, Discussion, Conclusion), where the segmentation problem is presented as a research problem that is investigated through the methods outlined and solved in the analysis. The discussion should connect to the wider literature to consider the approaches used by others to solve similar problems.

Second, a file (preferrably .py / .ipynb) containing the source code for your solution. This should be properly annotated and commented. For advice on how to correctly format code for delivery take a look [here](https://www.python.org/dev/peps/pep-0008/). It is important to provide frequent in-line comments to explain what your code is doing. Finally, if you use functions, they should be properly commented showing their input and output arguments as well as a brief description of the function.

**Deadline**

Both documents should be uploaded to moodle by **28th March 2018**.